# ¿¡Transformadores!?

## Atención

Motivado por los problemas de eficiencia computacional inherentes con las Redes Neuronales Recurrentes, en 2017 aparece la arquitectura de los Transformadores [1]. Con este modelo se logra paralelizar el proceso de entrenamiento capturando las secuencias con atención, al mismo tiempo que se codifica la posición de cada elemento en la secuencia.

> Los Transformadores se basan completamente en los mecanismos de auto-atención sin utilizar una arquitectura recurrente alineada en secuencia.

En lugar de calcular la atención una sola vez, los Transformadores utilizan un mecanismo de cabezas múltiples que ejecutan la auto-atención varias veces en **paralelo**, permitiendo que el modelo atienda de forma conjunta a información de diferentes subespacios de representación en **diferentes posiciones**. Al final, las salidas de atención se concatenan y se transforman linealmente en las dimensiones esperadas. 

<center>
<img src="https://d2l.ai/_images/multi-head-attention.svg" alt="Figure 10.3.3: https://d2l.ai/chapter_attention-mechanisms/transformer.html" width="50%">
<br>
<a href="https://d2l.ai/chapter_attention-mechanisms/transformer.html" target="_top">Multi-head attention</a> [2].
<br>
</center>
<br>


## Codificador

El codificador genera una representación basada en la atención, con la capacidad de localizar información específica del contexto [1].

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/5.1.PNG" alt="pesos" width="15%">
<br>
</center>

Contiene una pila de 6 capas idénticas (N). Cada capa tiene una capa de auto-atención de múltiples cabezas y cada subcapa tiene una conexión residual + capa de normalización. Todas las subcapas generan datos de la misma dimensión (512).

## Decodificador

El decodificador se encarga de recobrar la representación codificada [1].

<center>
<img src="http://www.cs.virginia.edu/~pc9za/riiaa_2020/5.2.PNG" alt="pesos" width="15%">
<br>
</center>

De igual manera, contiene una pila de 6 capas idénticas (N). Cada capa tiene dos subcapas de mecanismos de atención de múltiples cabezales y cada subcapa tiene una conexión residual + una capa de normalización.
Es importante destacar que en el decodificador, la primera subcapa de atención se modifica para evitar que las posiciones atiendan a posiciones posteriores (evitar ver al futuro).

# ¡Ajustemos un Transformador para clasificación de texto!

Vamos a trabajar con un [set de datos de noticias](https://archive.ics.uci.edu/ml/datasets/News+Aggregator) [3], las cuales vamos a clasificar en varias categorías. Trabajaremos con una version más pequeña del set original. Acá podrá descargar la versión modificada: [news_corpora_small.csv](http://www.cs.virginia.edu/~pc9za/riiaa_2020/news_corpora_small.csv)

Ajustaremos un modelo de la familia de los Transformadores para clasificar los titulares de noticias en 4 categorías.

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.cs.virginia.edu/~pc9za/riiaa_2020/news_corpora_small.csv', 'news_corpora_small.csv')

('news_corpora_small.csv', <http.client.HTTPMessage at 0x7f627837b1d0>)

## Instalemos Hugging Face
![face](https://huggingface.co/front/assets/huggingface_logo.svg)

Utilizaremos la librería de [Hugging Face](https://huggingface.co/).

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 890kB 15.6MB/s 
     |████████████████████████████████| 3.0MB 16.8MB/s 
     |████████████████████████████████| 1.1MB 31.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5c80c8bd9842b50378e7aac4f43937064fbd9549cbd1235f5ba8e4aac729a284
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## El modelo :: Bidirectional Encoder Representations from Transformers

Con Bidirectional Encoder Representations from Transformers (**BERT**), un modelo es pre-entrenado con datos que no requieren ser etiquetados. Una vez entrenado, el modelo genera una representación densa de la entrada. Para resolver otras tareas de PLN, como *clasificación de texto*, modificamos el modelo (e.g. agregando mas capas) y lo volvemos a entrenar con los datos (en este caso los titulares de las noticias) y sus respectivas etiquetas (nuestras 4 categorías).

Trabajaremos con una versión más pequeña de BERT: [DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter](https://arxiv.org/abs/1910.01108) [4].

In [ ]:
# importamos torch
import torch
from torch.utils.data import Dataset, DataLoader
# importamos la libreria de hugging face
import transformers
from transformers import DistilBertModel, DistilBertTokenizer
# importamos pandas
import pandas as pd

In [ ]:
# Importamos el csv en pandas
df = pd.read_csv('news_corpora_small.csv', sep=',', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# trabajaremos unicamente con los encabezados y las categorias
df = df[['TITLE','CATEGORY']]

# setear ids y descripciones de categorias
cat_to_id = {
    'e':0,
    'b':1,
    't':2,
    'm':3
}
letter_to_desc = {
    'e':'Entretenimiento',
    'b':'Beneficios',
    't':'Tecnologia',
    'm':'Medicina'
}
def add_cat_id(x):
    return cat_to_id[x]
df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: add_cat_id(x))

def update_cat(x):
    return letter_to_desc[x]
df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

id_to_desc = {
    0:'Entretenimiento',
    1:'Beneficios',
    2:'Tecnologia',
    3:'Medicina'
}

In [ ]:
print ("Tamano del set de datos: {}".format(len(df)))
df.head()

Tamano del set de datos: 40000


,TITLE,CATEGORY,ENCODE_CAT
0,Breaking Bad's Bryan Cranston helps student ge...,Entretenimiento,0
1,Is AC/DC retiring?,Entretenimiento,0
2,'Horrible Bosses 2' Trailer Shows A Desperate ...,Entretenimiento,0
3,Lupita Nyong'o named 'Most Beautiful' woman by...,Entretenimiento,0
4,6 TV Shows We'd Like To See Get The Big-Screen...,Entretenimiento,0


Dividimos el set de datos en dos subsets: uno de entrenamiento (75% de los datos) y otro de prueba (25% de los datos).

Definimos nuestro Dataset y Dataloader.

In [ ]:
class News_Headears(Dataset):
    def __init__(self, data, tokenizer, max_length=60):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.length = len(data)
        
    def __getitem__(self, index):
        inputs = self.tokenizer.encode_plus(self.data.TITLE[index],
                                            None,
                                            add_special_tokens=True,
                                            max_length=self.max_length,
                                            pad_to_max_length=True,
                                            return_token_type_ids=True,
                                            truncation=True)
        return torch.tensor(inputs['input_ids'], dtype=torch.long), torch.tensor(inputs['attention_mask'], dtype=torch.long), torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
    
    def __len__(self):
        return self.length

In [ ]:
# Definimos nuestro split
TRAIN_SPLIT_SIZE = 0.75
BATCH_SIZE = 16
train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 12 }
test_params = {'batch_size': BATCH_SIZE,
                'shuffle': False,
                'num_workers': 12 }

train_dataset = df.sample(frac=TRAIN_SPLIT_SIZE, random_state=0) # seteamos una semilla para obtener la misma particion posteriormente
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("Dataset de entrenamiento: {}".format(train_dataset.shape))
print("Dataset de prueba: {}".format(test_dataset.shape))

print ("\nNumero de datos correspondientes a cada categoria en nuestro split de entrenamiento:")
print (train_dataset['CATEGORY'].value_counts())
print ("\nNumero de datos correspondientes a cada categoria en nuestro split de prueba:")
print (test_dataset['CATEGORY'].value_counts())

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
train_data = News_Headears(train_dataset, tokenizer)
test_data = News_Headears(test_dataset, tokenizer)

training_loader = DataLoader(train_data, **train_params)
testing_loader = DataLoader(test_data, **test_params)

Dataset de entrenamiento: (30000, 3)
Dataset de prueba: (10000, 3)

Numero de datos correspondientes a cada categoria en nuestro split de entrenamiento:
Tecnologia         7543
Entretenimiento    7512
Beneficios         7493
Medicina           7452
Name: CATEGORY, dtype: int64

Numero de datos correspondientes a cada categoria en nuestro split de prueba:
Medicina           2548
Beneficios         2507
Entretenimiento    2488
Tecnologia         2457
Name: CATEGORY, dtype: int64


Ahora vamos a definir la arquitectura del modelo que vamos a utilizar para entrenar nuestra tarea de clasificacion.

Concatenaremos una capa lineal a DistillBERT y ajustaremos nuestro modelo.

BERT agrega una token llamado *CLS* al comienzo de cada oración para clasificación. Ese token se puede considerar como un embedding de toda la oración (por eso tomamos <code>output[0][:,0,:]</code> como entrada a nuestra capa lineal de clasificacion)


In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert_output = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.ll1 = torch.nn.Linear(768, 50)
        self.ll2 = torch.nn.Linear(50, 4)
    
    def forward(self, ids, mask):
        output = self.bert_output(ids, mask)
        output = torch.relu(self.ll1(output[0][:,0,:]))
        output = self.ll2(output)
        return output

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print ("Si tenemos un GPU disponible, usemoslo! - Device: {}\n".format(device))

model = Model()
model.to(device)
print (model)

Si tenemos un GPU disponible, usemoslo! - Device: cuda




Model(
  (bert_output): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

## Ajustemos el modelo

Utilizaremos CE como funcion de perdida y Adam como algoritmo optimizador

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-05)

Definamos nuestra funcion de entrenamiento

In [ ]:
def train(epoch):
    model.train()
    for i, (ids, mask, targets) in enumerate(training_loader):
        ids = ids.to(device)
        mask = mask.to(device)
        targets = targets.to(device)

        outputs = model(ids, mask)
        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if i%100 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
EPOCHS = 2
for epoch in range(0, EPOCHS):
  train(epoch)

Epoch: 0, Loss:  1.365579605102539
Epoch: 0, Loss:  1.3933652639389038
Epoch: 0, Loss:  1.3278939723968506
Epoch: 0, Loss:  1.3729243278503418
Epoch: 0, Loss:  1.2607274055480957
Epoch: 0, Loss:  0.952964723110199
Epoch: 0, Loss:  1.0547508001327515
Epoch: 0, Loss:  1.2827869653701782
Epoch: 0, Loss:  1.0623592138290405
Epoch: 0, Loss:  0.9535151124000549
Epoch: 0, Loss:  0.7681649923324585
Epoch: 0, Loss:  0.8218606114387512
Epoch: 0, Loss:  0.6384872198104858
Epoch: 0, Loss:  0.559400737285614
Epoch: 0, Loss:  0.4348406195640564
Epoch: 0, Loss:  0.9387041330337524
Epoch: 0, Loss:  0.295390784740448
Epoch: 0, Loss:  0.39555802941322327
Epoch: 0, Loss:  0.2449529618024826
Epoch: 1, Loss:  0.5813267230987549
Epoch: 1, Loss:  0.5266097784042358
Epoch: 1, Loss:  0.44435983896255493
Epoch: 1, Loss:  0.46799609065055847
Epoch: 1, Loss:  0.437944620847702
Epoch: 1, Loss:  0.6578884124755859
Epoch: 1, Loss:  0.7687342762947083
Epoch: 1, Loss:  0.6048207879066467
Epoch: 1, Loss:  0.51107162237

## Validemos el modelo

In [ ]:
import torch.nn.functional as F

n_correct = 0
total = 0

model.eval()
with torch.no_grad():
    for (ids, mask, targets) in testing_loader:
        ids = ids.to(device)
        mask = mask.to(device)
        targets = targets.to(device)

        outputs = model(ids, mask)
        softmax_outputs = F.softmax(outputs, dim=1)
        big_val, big_idx = torch.max(softmax_outputs.data, dim=1)
        
        total += len(targets)
        n_correct += (big_idx==targets).sum().item()

print ("Con tan solo {} epochs de entrenamiento, el accuracy de nuestro modelo es de: {}%".format(EPOCHS, (n_correct*100.0)/total))

Con tan solo 2 epochs de entrenamiento, el accuracy de nuestro modelo es de: 82.07%


Salvemos los parametros y pesos de nuestro para resumir el entrenamiento

In [ ]:
model_checkpoint = '{}_epochs_model.ckpt'.format(EPOCHS)
vocab_file = 'vocab.ckpt'

torch.save(model, model_checkpoint)
tokenizer.save_vocabulary(vocab_file)

print ("Modelo y vocabulario guardados")

Modelo y vocabulario guardados


Ahora probemos nuestro modelo con algun input real

In [ ]:
#@title Ingrese una oración para clasificarla en una de los cuatro categorias definidas. { display-mode: "form" }

input = 'Protect Your Privacy and the Environment While Upgrading Your Gear' #@param {type:"string"}

tokenized_input = tokenizer.encode_plus(input,
                                None,
                                add_special_tokens=True,
                                max_length=60,
                                pad_to_max_length=True,
                                return_token_type_ids=True,
                                truncation=True)

In [ ]:
outputs = model(torch.tensor(tokenized_input['input_ids'], dtype=torch.long).unsqueeze(0).cuda(), torch.tensor(tokenized_input['attention_mask'], dtype=torch.long).unsqueeze(0).cuda())
softmax_outputs = F.softmax(outputs, dim=1)
out_val, out_idx = torch.max(softmax_outputs.data, dim=1)
print ("Entrada: \"{}\" || Categoría: {}".format(input, id_to_desc[out_idx.item()]))

Entrada: "Protect Your Privacy and the Environment While Upgrading Your Gear" || Categoría: Tecnologia


## Ejercicios



1.   Modifique el modelo para obtener un mejor accuracy
2.   Cambie los hiper-parametros, por ejemplo: cambiar el learning rate, o el algoritmo de optimizacion (utilizar SGD en lugar de Adam)
3.   Dibuje las curvas de el loss y el accuracy durante el entrenamiento - utilizando el training set




# Referencias

[1] Vaswani, Ashish, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. "Attention is all you need." In Advances in neural information processing systems, pp. 5998-6008. 2017.

[2] D2l.ai. 2020. 10.3. Transformer — Dive Into Deep Learning 0.14.3 Documentation. [online] Disponible en: <https://d2l.ai/chapter_attention-mechanisms/transformer.html>. Accesado el 20 Agosto del 2020.

[3] Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

[4] Sanh, V., Debut, L., Chaumond, J., & Wolf, T. (2019). DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter. arXiv preprint arXiv:1910.01108.